In [ ]:
#                ETAPA 1:

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def preprocess_image(image_path, target_size=(768, 1024), clahe_params=(4, (16, 16))):
    """
    Preprocesar una imagen: convertir a escala de grises, redimensionar, y aplicar CLAHE.
    
    Parámetros:
        image_path (str): Ruta del archivo de imagen.
        target_size (tuple): Tamaño objetivo para redimensionar (ancho, alto).
        clahe_params (tuple): Parámetros de CLAHE (clipLimit, tileGridSize).
    
    Retorna:
        dict: Diccionario con las imágenes preprocesadas (original, gray, clahe).
    """
    # Validar si el archivo existe
    if not os.path.exists(image_path):
        print(f"Error: La imagen {image_path} no existe.")
        return None

    # Cargar la imagen
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: No se pudo cargar la imagen {image_path}.")
        return None

    try:
        # Convertir a escala de grises
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Redimensionar la imagen
        image_gray_resized = cv2.resize(image_gray, target_size, interpolation=cv2.INTER_AREA)
        image_color_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

        # Aplicar CLAHE
        clahe = cv2.createCLAHE(clipLimit=clahe_params[0], tileGridSize=clahe_params[1])
        image_clahe = clahe.apply(image_gray_resized)

        return {
            "original": image_color_resized,
            "gray": image_gray_resized,
            "clahe": image_clahe
        }

    except Exception as e:
        print(f"Error procesando la imagen {image_path}: {e}")
        return None


def preprocess_images_in_directory(image_dir, target_size=(768, 1024), clahe_params=(4, (16, 16))):
    """
    Preprocesar todas las imágenes en un directorio con validaciones robustas.
    
    Parámetros:
        image_dir (str): Directorio con imágenes.
        target_size (tuple): Tamaño objetivo para redimensionar (ancho, alto).
        clahe_params (tuple): Parámetros de CLAHE (clipLimit, tileGridSize).
    
    Retorna:
        dict: Diccionario con los resultados preprocesados (clave: nombre de archivo).
    """
    processed_images = {}
    problematic_images = []

    for filename in os.listdir(image_dir):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(image_dir, filename)
            result = preprocess_image(image_path, target_size, clahe_params)

            if result is not None:
                processed_images[filename] = result
            else:
                problematic_images.append(filename)

    print(f"Imágenes procesadas: {len(processed_images)}")
    print(f"Imágenes problemáticas: {len(problematic_images)}")
    if problematic_images:
        print("Lista de imágenes problemáticas:")
        print(problematic_images)

    return processed_images

print("listo sstage 1")
# Directorio de entrada
# image_dir = "./img"  # Cambia esta ruta según tu estructura de directorios

# # Parámetros
# target_size = (768, 1024)  # Dimensiones estándar
# clahe_params = (4, (16, 16))  # Parámetros de CLAHE

# Ejecutar el preprocesamiento
# processed_images = preprocess_images_in_directory(image_dir, target_size, clahe_params)

# # Visualizar una imagen preprocesada como ejemplo
# if processed_images:
#     example_filename = list(processed_images.keys())[0]
#     example_data = processed_images[example_filename]

#     plt.figure(figsize=(15, 5))
#     plt.subplot(1, 3, 1)
#     plt.imshow(cv2.cvtColor(example_data["original"], cv2.COLOR_BGR2RGB))
#     plt.title(f"Original ({example_filename})")
#     plt.axis("off")

#     plt.subplot(1, 3, 2)
#     plt.imshow(example_data["gray"], cmap="gray")
#     plt.title("Escala de grises")
#     plt.axis("off")

#     plt.subplot(1, 3, 3)
#     plt.imshow(example_data["clahe"], cmap="gray")
#     plt.title("CLAHE")
#     plt.axis("off")

#     plt.tight_layout()
#     plt.show()


listo sstage 1


In [ ]:
#                       ETAPA 2

In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def extract_roi(image_data, annotations, roi_size=44, visualize=False):
    """
    Extrae ROIs centradas en las anotaciones NUCLEUS y ajusta las coordenadas para cada ROI.

    Parámetros:
        image_data (dict): Diccionario con las imágenes procesadas ("original", "gray", "clahe").
        annotations (pd.DataFrame): DataFrame con las anotaciones para la imagen.
        roi_size (int): Tamaño de la ROI (cuadrada).
        visualize (bool): Si es True, muestra las visualizaciones de las ROIs.

    Retorna:
        list: Lista de ROIs extraídas (grises y color) y sus anotaciones ajustadas.
    """
    rois = []
    half_size = roi_size // 2

    # Filtrar anotaciones para NUCLEUS
    nucleus_annotations = annotations[annotations['structure'] == 'NUCLEUS']
    # kinetoplast_annotations = annotations[annotations['structure'] == 'KINETOPLAST']

    for _, row in nucleus_annotations.iterrows():
        x, y = int(row['new_x']), int(row['new_y'])

        # Definir límites de la ROI
        x_start, x_end = max(0, x - half_size), min(image_data['clahe'].shape[1], x + half_size)
        y_start, y_end = max(0, y - half_size), min(image_data['clahe'].shape[0], y + half_size)

        # Extraer ROIs
        roi_gray = image_data['clahe'][y_start:y_end, x_start:x_end]
        roi_color = image_data['original'][y_start:y_end, x_start:x_end, :]

        # Ajustar coordenadas para las anotaciones
        adjusted_annotations = annotations[
            (annotations['new_x'].between(x_start, x_end)) &
            (annotations['new_y'].between(y_start, y_end))
        ].copy()
        adjusted_annotations['adjusted_x'] = adjusted_annotations['new_x'] - x_start
        adjusted_annotations['adjusted_y'] = adjusted_annotations['new_y'] - y_start

        rois.append((roi_gray, roi_color, adjusted_annotations))

        # Visualización opcional
        if visualize:
            plt.figure(figsize=(15, 5))

            # Visualizar ROI en escala de grises
            plt.subplot(1, 2, 1)
            plt.imshow(roi_gray, cmap='gray', vmin=0, vmax=255)
            for _, anno in adjusted_annotations.iterrows():
                color = 'red' if anno['structure'] == 'NUCLEUS' else 'blue'
                plt.scatter(
                    anno['adjusted_x'],
                    anno['adjusted_y'],
                    color=color, label=anno['structure'], s=50
                )
            plt.title("ROI en escala de grises")
            plt.axis('off')
            plt.legend()

            # Visualizar ROI en color
            plt.subplot(1, 2, 2)
            plt.imshow(cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB))
            for _, anno in adjusted_annotations.iterrows():
                color = 'red' if anno['structure'] == 'NUCLEUS' else 'blue'
                plt.scatter(
                    anno['adjusted_x'],
                    anno['adjusted_y'],
                    color=color, label=anno['structure'], s=50
                )
            plt.title("ROI en color")
            plt.axis('off')
            plt.legend()

            plt.tight_layout()
            plt.show()

    return rois

def process_stage2(processed_images, annotations_csv_path, roi_size=44, visualize=False):
    """
    Procesa todas las imágenes y extrae ROIs según las anotaciones.

    Parámetros:
        processed_images (dict): Diccionario de imágenes preprocesadas.
        annotations_csv_path (str): Ruta al archivo CSV con las anotaciones.
        roi_size (int): Tamaño de las ROIs cuadradas.
        visualize (bool): Si es True, muestra las visualizaciones de las ROIs.

    Retorna:
        dict: Diccionario con las ROIs extraídas por imagen.
    """
    annotations = pd.read_csv(annotations_csv_path)
    all_rois = {}

    for image_name, image_data in processed_images.items():
        print(f"Procesando ROIs para la imagen: {image_name}")

        # Filtrar anotaciones para la imagen actual
        image_annotations = annotations[annotations['filename'] == image_name]

        # Extraer ROIs
        rois = extract_roi(image_data, image_annotations, roi_size=roi_size, visualize=visualize)
        all_rois[image_name] = rois

    print("Completado stage 2")
    return all_rois


print("Completado stage 2")
# Parámetros iniciales
# annotations_csv_path = "./ft_orig/supp_data01.csv"
# roi_size = 44

# # Ejecutar la etapa 2
# all_rois = process_stage2(processed_images, annotations_csv_path, roi_size=roi_size, visualize=False)

# Resultado: `all_rois` contiene todas las ROIs extraídas.


Completado stage 2


In [ ]:
#                   ETAPA 3

In [3]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def is_overlapping(roi1, roi2, margin=4):
    """
    Verifica si dos ROIs se superponen con un margen dado.
    Cada ROI se define como ((x_start, y_start), (x_end, y_end)).
    """
    # roi1: ((x1_start, y1_start), (x1_end, y1_end))
    # roi2: ((x2_start, y2_start), (x2_end, y2_end))
    return not (
        roi1[1][0] + margin <= roi2[0][0] or 
        roi1[0][0] - margin >= roi2[1][0] or 
        roi1[1][1] + margin <= roi2[0][1] or 
        roi1[0][1] - margin >= roi2[1][1]
    )

def generate_rois_no_parasite_gray(
    image_gray, 
    file_annotations, 
    roi_size=44, 
    margin=4, 
    max_attempts=1000
):
    """
    Generar ROIs de parásitos y NO-parásitos en escala de grises.
    - image_gray: np.ndarray (escala de grises, ya redimensionada).
    - file_annotations: DataFrame con las anotaciones (x,y) adecuadas a la misma escala.
    Retorna:
      parasite_rois: Lista de ROIs de parásitos.
      non_parasite_rois: Lista de ROIs de no-parásitos.
    """
    image_height, image_width = image_gray.shape
    parasite_rois = []
    non_parasite_rois = []

    # 1) Generar ROIs de parásitos a partir de las anotaciones
    for _, row in file_annotations.iterrows():
        x_center = int(row['new_x'])
        y_center = int(row['new_y'])
        # Cálculo de los bordes, evitando salirnos
        x_start = max(0, x_center - roi_size // 2)
        x_end   = min(image_width, x_center + roi_size // 2)
        y_start = max(0, y_center - roi_size // 2)
        y_end   = min(image_height, y_center + roi_size // 2)

        roi = ((x_start, y_start), (x_end, y_end))
        parasite_rois.append(roi)

    # 2) Generar ROIs de no-parásitos cerca de las ROIs de parásitos
    for parasite_roi in parasite_rois:
        # Tomamos el centro de la ROI de parásito para buscar aleatoriamente cerca
        x_center = (parasite_roi[0][0] + parasite_roi[1][0]) // 2
        y_center = (parasite_roi[0][1] + parasite_roi[1][1]) // 2

        attempts = 0
        while attempts < max_attempts:
            rand_x = x_center + np.random.randint(-50, 50)
            rand_y = y_center + np.random.randint(-50, 50)

            x_start = max(0, rand_x - roi_size // 2)
            x_end   = min(image_width, rand_x + roi_size // 2)
            y_start = max(0, rand_y - roi_size // 2)
            y_end   = min(image_height, rand_y + roi_size // 2)

            roi_proposed = ((x_start, y_start), (x_end, y_end))

            # Verificar que NO se solape con parásitos ni con otros no-parásitos
            overlap_with_parasites = any(is_overlapping(roi_proposed, r, margin) for r in parasite_rois)
            overlap_with_nonparas  = any(is_overlapping(roi_proposed, r, margin) for r in non_parasite_rois)

            if not overlap_with_parasites and not overlap_with_nonparas:
                non_parasite_rois.append(roi_proposed)
                break

            attempts += 1

    return parasite_rois, non_parasite_rois

def generate_rois_no_parasite_color_from_gray(
    image_color, 
    parasite_rois_gray, 
    non_parasite_rois_gray
):
    """
    Con las coordenadas de ROIs en gris, extraemos la versión a color correspondiente.
    Retorna dos listas: parasite_rois_color, non_parasite_rois_color, 
    donde cada elemento es ((x_start, y_start), (x_end, y_end), subimg_color).
    """
    parasite_rois_color = []
    non_parasite_rois_color = []

    for roi in parasite_rois_gray:
        (x_start, y_start), (x_end, y_end) = roi
        roi_color = image_color[y_start:y_end, x_start:x_end, :]
        parasite_rois_color.append((roi, roi_color))

    for roi in non_parasite_rois_gray:
        (x_start, y_start), (x_end, y_end) = roi
        roi_color = image_color[y_start:y_end, x_start:x_end, :]
        non_parasite_rois_color.append((roi, roi_color))

    return parasite_rois_color, non_parasite_rois_color

def visualize_rois_images(image_gray, image_color, parasite_rois, non_parasite_rois, roi_size=44):
    """
    Visualizar la imagen completa en gris y color, dibujando ROIs de parásitos y no-parásitos.
    """
    plt.figure(figsize=(15, 7))

    # 1) Imagen en gris
    plt.subplot(1, 2, 1)
    plt.imshow(image_gray, cmap='gray')
    plt.title('Imagen Gray con ROIs')
    ax1 = plt.gca()
    for roi in parasite_rois:
        x_start, y_start = roi[0]
        rect = plt.Rectangle((x_start, y_start), roi_size, roi_size,
                             linewidth=2, edgecolor='green', facecolor='none')
        ax1.add_patch(rect)
    for roi in non_parasite_rois:
        x_start, y_start = roi[0]
        rect = plt.Rectangle((x_start, y_start), roi_size, roi_size,
                             linewidth=2, edgecolor='red', facecolor='none')
        ax1.add_patch(rect)
    plt.axis('off')

    # 2) Imagen a color
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
    plt.title('Imagen Color con ROIs')
    ax2 = plt.gca()
    for roi in parasite_rois:
        x_start, y_start = roi[0]
        rect = plt.Rectangle((x_start, y_start), roi_size, roi_size,
                             linewidth=2, edgecolor='green', facecolor='none')
        ax2.add_patch(rect)
    for roi in non_parasite_rois:
        x_start, y_start = roi[0]
        rect = plt.Rectangle((x_start, y_start), roi_size, roi_size,
                             linewidth=2, edgecolor='red', facecolor='none')
        ax2.add_patch(rect)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

def process_stage3(
    processed_images, 
    annotations_csv_path,
    roi_size=44,
    margin=4,
    max_attempts=1000,
    visualize=False
):
    """
    Etapa 3 corregida:
    - Ya no re-lee las imágenes desde disco.
    - Usa las mismas imágenes preprocesadas (diccionario `processed_images`) 
      y el mismo CSV de anotaciones, con `new_x`, `new_y` compatibles.
    
    Retorna:
      stage3_results: Dict con la info de ROIs parásito / no-parásito 
                      en gris y color por cada imagen.
    """
    annotations_df = pd.read_csv(annotations_csv_path)
    stage3_results = {}

    # Iterar sobre las imágenes del pipeline (Etapa 1)
    for image_name, image_data in processed_images.items():
        # Filtrar anotaciones de la imagen actual
        file_annotations = annotations_df[annotations_df['filename'] == image_name]

        if file_annotations.empty:
            print(f"[Stage 3] Sin anotaciones para {image_name}, se omite.")
            continue

        # 1) Seleccionar la imagen en gris y en color 
        #    (con la misma escala del preprocesado).
        #    - 'gray' si quieres la versión redimensionada en gris (sin CLAHE)
        #    - 'clahe' si quieres la versión con CLAHE
        #    - 'original' es la versión a color redimensionada
        image_gray  = image_data['gray']   # o 'clahe', según necesites
        image_color = image_data['original']

        # 2) Generar ROIs parásito y no-parásito en gris
        parasite_rois_gray, non_parasite_rois_gray = generate_rois_no_parasite_gray(
            image_gray, file_annotations, roi_size=roi_size, margin=margin, max_attempts=max_attempts
        )

        # 3) Generar la versión color de esos mismos ROIs
        parasite_rois_color, non_parasite_rois_color = generate_rois_no_parasite_color_from_gray(
            image_color, parasite_rois_gray, non_parasite_rois_gray
        )

        # 4) (Opcional) Visualizar
        # if visualize:
        #     visualize_rois_images(image_gray, image_color, parasite_rois_gray, non_parasite_rois_gray, roi_size)

        # 5) Guardar el resultado en un diccionario
        stage3_results[image_name] = {
            "parasite_rois_gray": parasite_rois_gray,
            "non_parasite_rois_gray": non_parasite_rois_gray,
            "parasite_rois_color": parasite_rois_color,
            "non_parasite_rois_color": non_parasite_rois_color
        }

        print(f"[Stage 3] Procesada imagen: {image_name}")
        print(f"   - #ROIs parásito:     {len(parasite_rois_gray)}")
        print(f"   - #ROIs no-parásito:  {len(non_parasite_rois_gray)}")

    return stage3_results
print("Completado stage 3")

Completado stage 3


In [ ]:
#       ETAPA 4.1

In [5]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.segmentation import felzenszwalb

##############################################################################
#   Función Auxiliar: Recortar un sub-ROI y proyectar parches a coordenadas locales
##############################################################################
def visualize_sub_roi_extern(
    image_gray,
    roi_external,
    patch_global,
    title="ROI Externo (No-Parásito)"
):
    """
    Toma un ROI externo (definido por ((x_start, y_start),(x_end, y_end))) 
    y un parche 3x3 en coordenadas 'globales' de la misma imagen_gray,
    recorta la subimagen, transforma coords a locales y muestra.

    Parámetros:
    -----------
    image_gray : np.ndarray
        Imagen (o ROI grande) en grises de la que se extrae la parte 'externa'.
    roi_external : tuple
        ((x_start, y_start), (x_end, y_end)) que define el ROI externo.
    patch_global : list
        Lista de 9 tuplas (y, x, val) con coords globales en image_gray.
    title : str
        Título que se mostrará en la visualización.

    Retorna:
    --------
    (sub_roi, local_patch):
      - sub_roi: recorte [y_start:y_end, x_start:x_end] de image_gray
      - local_patch: parche con coords locales (restando y_start, x_start)
    """
    (x_start, y_start), (x_end, y_end) = roi_external

    # 1) Recortar la subimagen
    sub_roi = image_gray[y_start:y_end, x_start:x_end]

    # 2) Convertir coords globales -> coords locales
    local_patch = []
    for (py, px, val) in patch_global:
        y_loc = py - y_start
        x_loc = px - x_start
        # Validar que esté dentro de sub_roi
        if 0 <= y_loc < sub_roi.shape[0] and 0 <= x_loc < sub_roi.shape[1]:
            local_patch.append((y_loc, x_loc, val))

    # 3) Visualizar si el parche conserva 9 píxeles
    if len(local_patch) == 9:
        plt.figure(figsize=(4, 4))
        plt.imshow(sub_roi, cmap='gray')
        plt.title(title)
        for (yy, xx, _) in local_patch:
            plt.scatter(xx, yy, c='red', s=10)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print(f"[visualize_sub_roi_extern] Parche con coords locales incompleto (len={len(local_patch)}).")

    return sub_roi, local_patch


##############################################################################
#                       Función: extract_areas_from_external_rois
##############################################################################
def extract_areas_from_external_rois(
    rois,
    roi_gray,
    total_areas,
    roi_size=44,
    area_size=3
):
    """
    Extraer áreas de 3x3 píxeles de los ROIs 'no-parásitos' 
    (u otros) generados en la Etapa 3.
    
    Parámetros:
    -----------
    rois : list
        Lista de ROIs, donde cada ROI es ((x_start, y_start), (x_end, y_end)).
    roi_gray : np.ndarray
        Imagen (o subimagen) en escala de grises donde se ubican los ROIs.
    total_areas : int
        Número de parches 3x3 requeridos.
    roi_size : int
        Tamaño cuadrado de cada ROI (por defecto 44).
    area_size : int
        Tamaño del parche (3 para 3x3).

    Retorna:
    --------
    areas : list
        Lista de parches, cada parche es una lista de 9 tuplas (y, x, intensidad).
    external_rois_used : list
        Lista de ROIs de las que efectivamente se extrajeron parches.
    """
    areas = []
    external_rois_used = []

    for roi in rois:
        # Si ya alcanzamos la cantidad de parches requeridos, detenemos el bucle.
        if len(areas) >= total_areas:
            break

        (x_start, y_start), (x_end, y_end) = roi

        # Verificar si el ROI es demasiado pequeño para un parche 3x3.
        if (x_end - x_start) < area_size or (y_end - y_start) < area_size:
            # No cabe un parche de 3x3, pasamos al siguiente ROI.
            continue

        # Intentar hasta 10 veces obtener un parche válido.
        for _ in range(10):
            rand_x = np.random.randint(x_start + 1, x_end - area_size + 1)
            rand_y = np.random.randint(y_start + 1, y_end - area_size + 1)

            area_coords = []
            parche_valido = True

            for dy in range(area_size):
                for dx in range(area_size):
                    # Coordenadas (globales dentro de roi_gray)
                    yy = rand_y + dy
                    xx = rand_x + dx

                    try:
                        # Intentar acceder a roi_gray[yy, xx].
                        val = roi_gray[yy, xx]
                        area_coords.append((yy, xx, val))
                    except IndexError:
                        # Reportar y marcar como no válido este intento.
                        print(f"[extract_areas_from_external_rois] "
                              f"ÍNDICE FUERA DE RANGO: y={yy}, x={xx}, shape={roi_gray.shape}")
                        parche_valido = False
                        break  # Romper el bucle interno

                if not parche_valido:
                    break  # Romper el bucle externo también

            # Si después de recorrer los 3x3 píxeles el parche sigue siendo válido, lo agregamos.
            if parche_valido and len(area_coords) == area_size ** 2:
                areas.append(area_coords)
                external_rois_used.append(roi)
                break  # Salir del for _ in range(10)
            
            if len(area_coords) == area_size ** 2:
                print(f"[Debug] Parche válido extraído de ROI externo: {area_coords}")
            else:
                print(f"[Error] Parche inválido en ROI externo: {roi}, tamaño obtenido: {len(area_coords)}")

    return areas, external_rois_used




##############################################################################
#                          Segmentación e Identificación
##############################################################################
def segment_roi_with_felzenszwalb(roi_gray, scale=250, sigma=0.5, min_size=100):
    """ Segmenta la ROI (grises) con Felzenszwalb. """
    segments = felzenszwalb(roi_gray, scale=scale, sigma=sigma, min_size=min_size)
    return segments

def identify_segments_with_annotations(segments, annotations):
    """ Determina IDs de segmentos que tienen anotaciones (NUCLEUS, etc.). """
    annotated_segments = set()
    for _, row in annotations.iterrows():
        x, y = int(row['adjusted_x']), int(row['adjusted_y'])
        if 0 <= x < segments.shape[1] and 0 <= y < segments.shape[0]:
            annotated_segments.add(segments[y, x])
    return annotated_segments


##############################################################################
#            Extracción de parches 3x3 (parásito y no-parásito local)
##############################################################################
def extract_3x3_around_annotation(roi_gray, annotations):
    """
    Extrae un parche 3x3 por cada anotación => parches "parásito".
    Retorna una lista de 9 tuplas (y,x,val) por parche.
    """
    parasite_areas_gray = []
    for _, row in annotations.iterrows():
        x_c = int(row['adjusted_x'])
        y_c = int(row['adjusted_y'])
        patch = []
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                yy, xx = y_c + dy, x_c + dx
                if 0 <= yy < roi_gray.shape[0] and 0 <= xx < roi_gray.shape[1]:
                    patch.append((yy, xx, roi_gray[yy, xx]))
        if len(patch) == 9:
            parasite_areas_gray.append(patch)
    return parasite_areas_gray


def extract_areas_of_3x3_gray(roi_gray, segments, annotated_segments, total_areas):
    """
    Extrae parches 3x3 "no-parásito" local, evitando segmentos con anotaciones.
    Intenta extraer 'total_areas'.
    """
    areas = []
    visited_pixels = set()
    h, w = segments.shape

    for seg_id in np.unique(segments):
        if seg_id in annotated_segments:
            continue

        pix_coords = np.argwhere(segments == seg_id)
        np.random.shuffle(pix_coords)

        for (py, px) in pix_coords:
            if py-1 < 0 or py+1 >= h or px-1 < 0 or px+1 >= w:
                continue

            patch_3x3 = []
            for dy in range(-1, 2):
                for dx in range(-1, 2):
                    yy, xx = py+dy, px+dx
                    patch_3x3.append((yy, xx, roi_gray[yy, xx]))

            # Comprobar segmento anotado / píxeles ya usados
            if any(segments[yy, xx] in annotated_segments for (yy, xx, _) in patch_3x3):
                continue
            if any((yy, xx) in visited_pixels for (yy, xx, _) in patch_3x3):
                continue

            areas.append(patch_3x3)
            visited_pixels.update([(yy, xx) for (yy, xx, _) in patch_3x3])

            if len(areas) >= total_areas:
                return areas

    return areas


##############################################################################
#                    Convertir parches 3x3 en gris a color
##############################################################################
def extract_same_3x3_in_color(roi_color, gray_patches):
    """
    Convierte parches locales de gris a color.

    Parámetros:
    -----------
    roi_color : np.ndarray
        Sub-ROI en color.
    gray_patches : list
        Lista de parches 3x3 en gris.

    Retorna:
    --------
    color_patches : list
        Lista de parches 3x3 en color.
    """
    color_areas = []
    for patch in gray_patches:
        c_patch = []
        for (yy, xx, _) in patch:
            if 0 <= yy < roi_color.shape[0] and 0 <= xx < roi_color.shape[1]:
                c_patch.append((yy, xx, roi_color[yy, xx]))
        color_areas.append(c_patch)
    return color_areas


##############################################################################
#   Función: Extraer Parches Externos en Color
##############################################################################
def extract_same_3x3_in_color_external(image_color_full, gray_patches_external, external_rois_used):
    """
    Convierte parches externos de gris a color dentro de las sub-ROIs externas.

    Parámetros:
    -----------
    image_gray_full : np.ndarray
        Imagen completa en color.
    gray_patches_external : list
        Lista de parches 3x3 en gris (coordenadas globales).
    external_rois_used : list
        Lista de ROIs externos ((x_start, y_start), (x_end, y_end)).

    Retorna:
    --------
    color_patches_external : list
        Lista de parches 3x3 en color dentro de las sub-ROIs externas.
    """
    color_patches_external = []

    for patch_3x3, roi_ext in zip(gray_patches_external, external_rois_used):
        (x_start, y_start), (x_end, y_end) = roi_ext
        roi_color_external = image_color_full[y_start:y_end, x_start:x_end]  # Sub-ROI en color

        patch_color_local = []
        for (py, px, _) in patch_3x3:
            y_loc = py - y_start
            x_loc = px - x_start
            if 0 <= y_loc < roi_color_external.shape[0] and 0 <= x_loc < roi_color_external.shape[1]:
                # Acceder al valor RGB/BGR completo
                patch_color_local.append((y_loc, x_loc, roi_color_external[y_loc, x_loc]))
        color_patches_external.append(patch_color_local)

    return color_patches_external


##############################################################################
#                Proceso central 4.1 para UNA ROI específica
##############################################################################
def process_pipeline_4_1(
    roi_gray,
    roi_color,
    adjusted_annotations,
    external_rois=None,
    visualize=False,
    image_gray_full=None, # Imagen global gris
    image_color_full=None # Imagen global color
):
    try:
        # A) Segmentar
        segments = segment_roi_with_felzenszwalb(roi_gray)
        annotated_segments = identify_segments_with_annotations(segments, adjusted_annotations)
        total_areas = len(adjusted_annotations)

        # B) Parches parásito (gris)
        parasite_areas_gray = extract_3x3_around_annotation(roi_gray, adjusted_annotations)


        # C) Parches No-Parásito Locales (Gris)
        non_parasite_areas_gray_local = extract_areas_of_3x3_gray(
            roi_gray, segments, annotated_segments, total_areas
        )

        # D) Parches No-Parásito Externos (Gris)
        non_parasite_areas_gray_external = []
        external_rois_used = []
        needed = total_areas - len(non_parasite_areas_gray_local)
        if external_rois is not None and needed > 0:
            non_parasite_areas_gray_external, external_rois_used = extract_areas_from_external_rois(
                rois=external_rois,
                roi_gray=image_gray_full,
                total_areas=needed,
                roi_size=44,
                area_size=3
            )
            # Si faltaron parches y se usaron ROIs externos, visualizamos los externos
            # if visualize and len(non_parasite_areas_gray_external) > 0:
            #     print(f"[Info] Visualizando ROIs externos usados: {len(external_rois_used)}")
        
            #     # Mostrar cada ROI externo utilizado
            #     for i, (roi_ext, patch_3x3) in enumerate(zip(external_rois_used, non_parasite_areas_gray_external)):
            #         visualize_external_roi_with_patches(
            #             image_gray_full=image_gray_full,  # Imagen completa en gris
            #             roi_external=roi_ext,  # Coordenadas del ROI externo
            #             patch_global=patch_3x3,  # Parches extraídos
            #             title=f"ROI Externo #{i+1} - Parches No-Parásito (Gris)"
            #         )

        # E) Convertir a Color
        parasite_areas_color = extract_same_3x3_in_color(roi_color, parasite_areas_gray)
        non_parasite_areas_color_local = extract_same_3x3_in_color(roi_color, non_parasite_areas_gray_local)
        non_parasite_areas_color_external = extract_same_3x3_in_color_external(
            image_color_full=image_color_full,
            gray_patches_external=non_parasite_areas_gray_external,
            external_rois_used=external_rois_used
        )
                
        # F) Visualizar (Opcional)
        # if visualize:
        #     visualize_roi_results(
        #         roi_gray=roi_gray,
        #         parasite_patches=parasite_areas_gray,
        #         non_parasite_patches_local=non_parasite_areas_gray_local,
        #         segments=segments
        #     )

        # Retornar Resultados
        return {
            "parasite_areas_gray": parasite_areas_gray,
            "non_parasite_areas_gray_local": non_parasite_areas_gray_local,
            "non_parasite_areas_gray_external": non_parasite_areas_gray_external,
            "parasite_areas_color": parasite_areas_color,
            "non_parasite_areas_color_local": non_parasite_areas_color_local,
            "non_parasite_areas_color_external": non_parasite_areas_color_external,
            "external_rois_used": external_rois_used,
            "segments": segments
        }

    except Exception as e:
        print(f"[process_pipeline_4_1] Error: {e}")
        return None



##############################################################################
#   Función Auxiliar: Visualizar Resultados
##############################################################################
def visualize_roi_results(roi_gray, parasite_patches, non_parasite_patches_local, segments):
    """
    Visualiza los resultados de la segmentación y parches de una ROI, incluyendo
    parches parásitos y no-parásitos locales.

    Parámetros:
    -----------
    roi_gray : np.ndarray
        Sub-ROI en escala de grises.
    parasite_patches : list
        Lista de parches parásito.
    non_parasite_patches_local : list
        Lista de parches no-parásito locales.
    segments : np.ndarray
        Segmentos obtenidos de la segmentación.
    """
    plt.figure(figsize=(12, 6))
    
    # Visualización de la segmentación
    plt.subplot(1, 2, 1)
    plt.imshow(segments, cmap='nipy_spectral')
    plt.title("Segmentación (Felzenszwalb)")
    plt.axis('off')
    
    # Mostrar parches parásitos en la segmentación
    for patch in parasite_patches:
        for (yy, xx, _) in patch:
            plt.scatter(xx, yy, c='green', s=10, label="Parásito")

    # Mostrar parches no-parásito locales en la segmentación
    for patch in non_parasite_patches_local:
        for (yy, xx, _) in patch:
            plt.scatter(xx, yy, c='red', s=10, label="No-Parásito Local")

    # Evitar etiquetas repetidas en la leyenda
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), loc="upper right", fontsize=8)

    # Visualización de la ROI en escala de grises con parches
    plt.subplot(1, 2, 2)
    plt.imshow(roi_gray, cmap='gray')
    plt.title("Parches 3x3 en ROI (Gris)")
    
    # Mostrar parches parásitos (verde)
    for patch in parasite_patches:
        for (yy, xx, _) in patch:
            plt.scatter(xx, yy, c='green', s=10)

    # Mostrar parches no-parásito locales (rojo)
    for patch in non_parasite_patches_local:
        for (yy, xx, _) in patch:
            plt.scatter(xx, yy, c='red', s=10)

    plt.axis('off')
    plt.tight_layout()
    plt.show()


def print_patch_values(results_4_1):
    """
    Imprime los valores de los parches (parásito y no-parásito)
    en gris y color para cada ROI procesada.

    Parámetros:
    -----------
    results_4_1 : dict
        Resultados del pipeline 4.1 por cada imagen.

    Retorna:
    --------
    None
    """
    for img_name, rois_results in results_4_1.items():
        print(f"\n=== Imagen {img_name} ===")
        for roi_idx, roi_data in enumerate(rois_results, start=1):
            print(f"--- ROI #{roi_idx} ---")
            print("Parches Parásito (Gris):")
            for patch in roi_data["parasite_areas_gray"]:
                print(patch)
            print("Parches No-Parásito (Gris - Locales):")
            for patch in roi_data["non_parasite_areas_gray_local"]:
                print(patch)
            print("Parches No-Parásito (Gris - Externos):")
            for patch in roi_data["non_parasite_areas_gray_external"]:
                print(patch)
            print("Parches Parásito (Color):")
            for patch in roi_data["parasite_areas_color"]:
                print(patch)
            print("Parches No-Parásito (Color - Locales):")
            for patch in roi_data["non_parasite_areas_color_local"]:
                print(patch)
            print("Parches No-Parásito (Color - Externos):")
            for patch in roi_data["non_parasite_areas_color_external"]:
                print(patch)



def visualize_external_roi_with_patches(
    image_gray_full,
    roi_external,
    patch_global,
    title="ROI Externo (No-Parásito)"
):
    """
    Visualiza un 'ROI externo' y el parche (o parches) 3x3 en coordenadas globales dentro de esa región.

    Parámetros:
    -----------
    image_gray_full : np.ndarray
        Imagen global (o subimagen grande) en escala de grises donde 'roi_external' es válido.
    roi_external : tuple
        ((x_start, y_start), (x_end, y_end)) que define el bounding box global.
    patch_global : list
        Lista con 9 tuplas (y, x, val) en coords globales, correspondiente al parche 3x3.
        (Si tienes varios parches, puedes pasar una lista de listas).
    title : str
        Título para la visualización.

    Retorna:
    --------
    None
        Solo muestra la figura en pantalla.
    """


    (x_start, y_start), (x_end, y_end) = roi_external

    # 1) Recortar la sub-ROI de la imagen global
    sub_roi = image_gray_full[y_start:y_end, x_start:x_end]

    # 2) Convertir coords globales -> coords locales
    #    (Por si solo tienes un parche, lo metemos en lista de listas para unificar)
    if len(patch_global) > 0 and isinstance(patch_global[0], tuple) and len(patch_global[0]) == 3:
        # Significa que patch_global es UNA sola lista de 9 tuplas
        patch_global = [patch_global]

    plt.figure(figsize=(5, 5))
    plt.imshow(sub_roi, cmap='gray')
    plt.title(title)

    colors = ["red", "blue", "green", "magenta"]  # por si quieres varios parches
    for idx, single_patch in enumerate(patch_global):
        color_p = colors[idx % len(colors)]
        local_coords = []
        for (py, px, _) in single_patch:
            y_loc = py - y_start
            x_loc = px - x_start
            # Solo pintar si cae dentro del sub_roi
            if 0 <= y_loc < sub_roi.shape[0] and 0 <= x_loc < sub_roi.shape[1]:
                local_coords.append((y_loc, x_loc))
        # Pintar cada píxel en color
        for (yl, xl) in local_coords:
            plt.scatter(xl, yl, c=color_p, s=20)

    plt.axis('off')
    plt.tight_layout()
    plt.show()



##############################################################################
#       Pipeline unificador: llama a Etapa 2, Etapa 3 y luego process_pipeline_4_1
##############################################################################
def pipeline_stage_4_1(processed_images, annotations_csv_path, roi_size=44, visualize=False):
    """
    Pipeline que integra:
      - Etapa 2 (process_stage2) para extraer ROIs,
      - Etapa 3 (process_stage3) para generar no-parásitos a nivel de imagen,
      - y finalmente la lógica central de Etapa 4.1 (process_pipeline_4_1)
        en cada ROI devuelta por Etapa 2.

    Retorna un dict con los resultados de la Etapa 4.1 por cada imagen.
    """
    # IMPORTAR TUS ETAPAS 2 Y 3 (ajusta si las tienes en celdas previas):
    # from stage2_module import process_stage2
    # from stage3_module import process_stage3

    # O si ya las definiste en celdas anteriores, sólo llámalas:
    # process_stage2(...)
    # process_stage3(...)

    rois_stage2 = process_stage2(
        processed_images=processed_images,
        annotations_csv_path=annotations_csv_path,
        roi_size=roi_size,
        visualize=False
    )

    stage3_results = process_stage3(
        processed_images=processed_images,
        annotations_csv_path=annotations_csv_path,
        roi_size=roi_size,
        margin=4,
        max_attempts=1000,
        visualize=False
    )

    all_results = {}
    for image_name, rois_list in rois_stage2.items():
        image_output = []

        # Extraer la lista de ROIs no-parásito a nivel de imagen
        external_rois_gray = None
        if image_name in stage3_results:
            external_rois_gray = stage3_results[image_name]["non_parasite_rois_gray"]

        for (roi_gray, roi_color, adjusted_annotations) in rois_list:
            if roi_gray is None or roi_color is None or adjusted_annotations.empty:
                print(f"[pipeline_stage_4_1] ROI inválida en {image_name}, se omite.")
                continue

            # Llamada al proceso final
            result_4_1 = process_pipeline_4_1(
                roi_gray=roi_gray,
                roi_color=roi_color,
                adjusted_annotations=adjusted_annotations,
                external_rois=external_rois_gray,
                visualize=visualize,
                image_gray_full=processed_images[image_name]["clahe"],  # <--- imagen global, si la tienes
                image_color_full=processed_images[image_name]["original"] 
            )

            if result_4_1:
                image_output.append(result_4_1)

        all_results[image_name] = image_output

    return all_results


##############################################################################
#                Ejemplo de uso final (opcional)
##############################################################################
# def example_run_stage4_1():
#     """
#     Demostración de cómo integrar:
#       - Etapa 1 => processed_images
#       - pipeline_stage_4_1 => llama a Etapa 2 y 3 internamente,
#         luego hace la lógica 4.1.
#     """
#     # from stage1_module import preprocess_images_in_directory  # Si lo necesitas

#     image_dir = "./img"
#     annotations_csv = "./ft_orig/supp_data01.csv"

#     # Ejemplo: suponer que ya tienes 'processed_images' de la Etapa 1:
#     # processed_images = preprocess_images_in_directory(image_dir, target_size=(768, 1024))

#     results_4_1 = pipeline_stage_4_1(
#         processed_images,      # Diccionario de la Etapa 1
#         annotations_csv,
#         roi_size=44,
#         visualize=True
#     )
    
#     for img_name, rois_results in results_4_1.items():
#         print(f"\n=== Imagen {img_name} ===")
#         print(f"   # de ROIs procesadas en 4.1: {len(rois_results)}")
#         for i, r in enumerate(rois_results, start=1):
#             print(f"   -> ROI #{i}: #ParchesParásito={len(r['parasite_areas_gray'])}, "
#                   f"#ParchesNoParásito (Locales)={len(r['non_parasite_areas_gray_local'])}, "
#                   f"#ParchesNoParásito (Externos)={len(r['non_parasite_areas_gray_external'])}")

#     # Imprimir valores en consola
#     #print_patch_values(results_4_1)
#     print("Termino stage 4.1")
# # Si deseas ejecutarlo directamente:
# example_run_stage4_1()

print("Completado stage 4.1")


Completado stage 4.1


In [ ]:
#            ETAPA 4.2

In [ ]:
import numpy as np
import cv2
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from mahotas.features import haralick
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pywt


# ==========================
# 1. Definiciones de features (igual que antes)
# ==========================

def calculate_intensity_features(area_3x3):
    """
    Calcula características de intensidad para un área 3x3 (array 2D).
    """
    area = np.array(area_3x3, dtype=float)  # por si acaso
    mean_intensity = np.mean(area)
    std_intensity  = np.std(area)
    min_intensity  = np.min(area)
    max_intensity  = np.max(area)

    # Evitar división por cero en kurtosis/skewness
    if std_intensity < 1e-12:
        std_intensity = 1e-12

    kurtosis = (np.sum((area - mean_intensity)**4) / area.size) / (std_intensity**4)
    skewness = (np.sum((area - mean_intensity)**3) / area.size) / (std_intensity**3)

    # Entropía aproximada
    entropy = -np.sum((area / 255) * np.log2((area / 255) + 1e-9))

    contrast = max_intensity - min_intensity

    return {
        "mean": mean_intensity,
        "std": std_intensity,
        "min": min_intensity,
        "max": max_intensity,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "entropy": entropy,
        "contrast": contrast,
    }
# ==========================
# Definiciones de Features Basados en Gradiente
# ==========================
def calculate_gradient_features(area_3x3):
    grad_x = cv2.Sobel(area_3x3, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(area_3x3, cv2.CV_64F, 0, 1, ksize=3)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    grad_orientation = np.arctan2(grad_y, grad_x)

    return {
        "gradient_mean_magnitude": np.mean(grad_magnitude),
        "gradient_std_magnitude": np.std(grad_magnitude),
        "gradient_mean_orientation": np.mean(grad_orientation),
    }


# ==========================
# Definiciones de Features Basados en Contornos
# ==========================
def calculate_contour_features(area_3x3):
    _, binary_area = cv2.threshold(area_3x3, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_area, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return {
        "contour_count": len(contours),
        "contour_area": sum(cv2.contourArea(c) for c in contours),
    }



# ==========================
# Definiciones de Features Basados en Histogramas de Color
# ==========================
def calculate_color_histogram_features(color_patch):
    """
    Calcula histogramas de color para un parche de 3x3 píxeles en color.
    color_patch: lista de 9 tuplas (y, x, [B, G, R])
    Retorna un diccionario con los histogramas de cada canal (B, G, R) como características individuales.
    """
    # Convertir la lista de tuplas en un array de NumPy con forma (3, 3, 3)
    color_array = np.array([item[2] for item in color_patch])  # Extraer [B, G, R] de cada tupla
    color_array = color_array.reshape(3, 3, 3)  # Reformar a (3, 3, 3)

    # Verificar que el array tenga la forma correcta
    if color_array.shape != (3, 3, 3):
        raise ValueError(f"Forma incorrecta del parche de color: {color_array.shape}")

    # Calcular histogramas para cada canal (B, G, R)
    hist_b = cv2.calcHist([color_array], [0], None, [8], [0, 256]).flatten()
    hist_g = cv2.calcHist([color_array], [1], None, [8], [0, 256]).flatten()
    hist_r = cv2.calcHist([color_array], [2], None, [8], [0, 256]).flatten()

    # Convertir los histogramas en características individuales
    color_hist_features = {}
    for i, value in enumerate(hist_b):
        color_hist_features[f"hist_b_{i}"] = value
    for i, value in enumerate(hist_g):
        color_hist_features[f"hist_g_{i}"] = value
    for i, value in enumerate(hist_r):
        color_hist_features[f"hist_r_{i}"] = value

    return color_hist_features


# ==========================
# Definiciones de Features Basados en Transformada de Gabor
# ==========================
def calculate_gabor_features(area_3x3):
    kernels = [cv2.getGaborKernel((3, 3), 4.0, theta, 8.0, 1.0, 0, ktype=cv2.CV_32F) for theta in np.linspace(0, np.pi, 4)]
    responses = [cv2.filter2D(area_3x3, cv2.CV_8UC3, k) for k in kernels]
    return {
        f"gabor_mean_{i}": np.mean(resp) for i, resp in enumerate(responses)
    }


def calculate_hog_features(area_3x3):
    """
    Calcula características HOG y las devuelve como características individuales.
    """
    hog = cv2.HOGDescriptor((3, 3), (2, 2), (1, 1), (1, 1), 9)
    hog_feats = hog.compute(area_3x3)
    
    # Convertir los features HOG en características individuales
    hog_features = {f"hog_feat_{i}": value for i, value in enumerate(hog_feats.flatten())}
    return hog_features

def calculate_glcm_features(area_3x3, distances=[1], angles=[0]):
    """
    Calcula características GLCM para un área 3x3 (array 2D).
    """
    area = np.array(area_3x3, dtype=float)
    amin, amax = area.min(), area.max()
    if amax == amin:
        area_normalized = np.zeros_like(area, dtype=np.uint8)
    else:
        area_normalized = ((area - amin) * 255.0 / (amax - amin)).astype(np.uint8)

    glcm = graycomatrix(area_normalized,
                        distances=distances,
                        angles=angles,
                        symmetric=True,
                        normed=True)

    contrast     = graycoprops(glcm, 'contrast')[0, 0]
    correlation  = graycoprops(glcm, 'correlation')[0, 0]
    homogeneity  = graycoprops(glcm, 'homogeneity')[0, 0]
    energy       = graycoprops(glcm, 'energy')[0, 0]

    return {
        "contrast": contrast,
        "correlation": correlation,
        "homogeneity": homogeneity,
        "energy": energy,
    }

def calculate_lbp_features(area_3x3, radius=1, n_points=8):
    """
    Calcula LBP (uniform) y retorna el histograma como características individuales.
    area_3x3: array 2D de 3x3.
    """
    area = np.array(area_3x3, dtype=float)
    lbp = local_binary_pattern(area, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)
    
    # Convertir el histograma en características individuales
    lbp_features = {f"lbp_hist_{i}": value for i, value in enumerate(lbp_hist)}
    return lbp_features


def validate_patch(patch):
    """
    Valida que el parche sea adecuado para el procesamiento.
    - patch: array numpy de 3x3 con valores de intensidad.

    Retorna True si el parche es válido, de lo contrario False.
    """
    print(f"[DEBUG] Validando parche: {patch}")
    if np.max(patch) == np.min(patch):
        print("[WARNING] Parche uniforme detectado. Saltando el cálculo.")
        return False
    # if np.max(patch) - np.min(patch) < 10:  # Ajusta el umbral según tus datos
    #     print("[WARNING] Rango de valores muy pequeño en el parche. Saltando el cálculo.")
    #     return False
    return True



def calculate_haralick_features(area_3x3):
    """
    Calcula características de Haralick para un parche dado.
    - area_3x3: array numpy de 3x3 con valores de intensidad.

    Retorna un diccionario con las características.
    """
    # Asegurar que el área no sea vacía
    if area_3x3.size == 0:
        print("[WARNING] Parche vacío detectado. Asignando valores predeterminados.")
        return {f"haralick_{i+1}": 0 for i in range(13)}

    amin, amax = np.min(area_3x3), np.max(area_3x3)
    
    # Normalizar el parche y agregar ruido si el rango es muy pequeño
    if amin == amax or (amax - amin) < 5:  # Ajusta el umbral según tus datos
        print("[INFO] Parche con bajo rango detectado. Agregando ruido.")
        area_normalized = (area_3x3 + np.random.uniform(-0.5, 0.5, area_3x3.shape)).astype(np.uint8)
    else:
        area_normalized = ((area_3x3 - amin) * 255.0 / (amax - amin)).astype(np.uint8)

    try:
        har = haralick(area_normalized, ignore_zeros=True).mean(axis=0)
        return {f"haralick_{i+1}": val for i, val in enumerate(har)}
    except ValueError as e:
        print(f"[ERROR] Fallo en cálculo de Haralick: {e}")
        return {f"haralick_{i+1}": 0 for i in range(13)}


def calculate_geometry_features(area_3x3):
    """
    Calcula características geométricas para un área 3x3 (array 2D).
    """
    area = np.array(area_3x3, dtype=float)
    amin, amax = area.min(), area.max()
    if amax == amin:
        area_normalized = np.zeros_like(area, dtype=np.uint8)
    else:
        area_normalized = ((area - amin) * 255.0 / (amax - amin)).astype(np.uint8)

    _, binary_area = cv2.threshold(area_normalized, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_area, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    moments = cv2.moments(binary_area)

    area_size = moments["m00"]
    perimeter = 0.0
    if len(contours) > 0:
        perimeter = cv2.arcLength(contours[0], True)

    aspect_ratio = moments["mu20"] / (moments["mu02"] + 1e-5)
    compacity    = (perimeter**2) / (4*np.pi*area_size + 1e-5)
    eccentricity = ((moments["mu20"] - moments["mu02"])**2 /
                    (moments["mu20"] + moments["mu02"] + 1e-5))
    roundness    = (4*np.pi*area_size) / (perimeter**2 + 1e-5)

    return {
        "aspect_ratio": aspect_ratio,
        "compacity": compacity,
        "eccentricity": eccentricity,
        "area": area_size,
        "perimeter": perimeter,
        "roundness": roundness,
    }

# ==========================
# 2. Nuevas funciones para color
# ==========================
def calculate_color_features_from_patch(color_patch):
    """
    color_patch: lista de 9 tuplas (y, x, [B, G, R])
    Retorna dict con r_mean, g_mean, b_mean, r_std, g_std, b_std
    """
    if len(color_patch) != 9:
        return {}
    # Extraemos [B,G,R] (shape = (9,3))
    pixels_bgr = [p[2] for p in color_patch]  # [B,G,R]
    pixels_bgr = np.array(pixels_bgr)
    b_mean, g_mean, r_mean = np.mean(pixels_bgr, axis=0)
    b_std, g_std, r_std    = np.std(pixels_bgr, axis=0)

    return {
        "r_mean": r_mean,  "g_mean": g_mean,  "b_mean": b_mean,
        "r_std":  r_std,   "g_std":  g_std,   "b_std":  b_std,
    }


def calculate_fft_features(area_3x3):
    """
    Calcula características de frecuencia (FFT) en un parche 3x3.
    Devuelve:
      - fft_energy: Suma de |FFT|^2
      - fft_peak_val: Valor del pico más alto en la magnitud FFT
      - fft_peak_idx_f, fft_peak_idx_c: Índices (fila, columna) del pico más prominente
      - fft_n_peaks: Número de picos por encima de un threshold
    """
    # Verificar si el parche tiene baja variabilidad
    if np.max(area_3x3) == np.min(area_3x3):
        return {
            "fft_energy": 0.0,
            "fft_peak_val": 0.0,
            "fft_peak_idx_f": 0,
            "fft_peak_idx_c": 0,
            "fft_n_peaks": 0
        }

    # 1. FFT 2D
    fft_2d = np.fft.fft2(area_3x3)
    fft_shifted = np.fft.fftshift(fft_2d)

    # 2. Magnitud
    magnitude = np.abs(fft_shifted)

    # 3. Energía total
    fft_energy = np.sum(magnitude**2)

    # 4. Componente principal (peak)
    # Evitar incluir el valor central (DC) si no aporta información
    magnitude[magnitude.shape[0] // 2, magnitude.shape[1] // 2] = 0
    peak_idx = np.unravel_index(np.argmax(magnitude), magnitude.shape)
    fft_peak_val = magnitude[peak_idx]

    # 5. Número de “picos” por encima de un threshold
    threshold = 0.5 * fft_peak_val
    fft_n_peaks = np.sum(magnitude >= threshold)

    # Manejar casos donde el pico más alto es insignificante
    if fft_peak_val < 1e-6:
        return {
            "fft_energy": float(fft_energy),
            "fft_peak_val": 0.0,
            "fft_peak_idx_f": 0,
            "fft_peak_idx_c": 0,
            "fft_n_peaks": 0
        }

    return {
        "fft_energy": float(fft_energy),
        "fft_peak_val": float(fft_peak_val),
        "fft_peak_idx_f": peak_idx[0],
        "fft_peak_idx_c": peak_idx[1],
        "fft_n_peaks": int(fft_n_peaks)
    }



def calculate_wavelet_features(area_3x3, wavelet='haar', level=1):
    """
    Calcula coeficientes wavelet de un parche 3x3.
    Retorna algunos coeficientes de sub-bandas (aprox, detalles).
    """
    area = np.array(area_3x3, dtype=float)
    
    # pywt.dwt2 => descomposición en 1 nivel
    # cA => aprox, (cH, cV, cD) => detalles
    cA, (cH, cV, cD) = pywt.dwt2(area, wavelet=wavelet)

    # Podrías extraer la energía de cada sub-banda
    # Por ej:
    cA_energy = np.sum(cA**2)
    cH_energy = np.sum(cH**2)
    cV_energy = np.sum(cV**2)
    cD_energy = np.sum(cD**2)

    # O podrías devolverlos “tal cual” si deseas
    # El shape de cA, cH, cV, cD dependerá de la wavelet y del nivel
    # Con 3×3, cA será 2×2, cH=2×2, etc.

    return {
        "wavelet_cA_energy": cA_energy,
        "wavelet_cH_energy": cH_energy,
        "wavelet_cV_energy": cV_energy,
        "wavelet_cD_energy": cD_energy
    }




# ==========================
# 3. Función principal de extracción de features en un parche
# ==========================
def extract_features_for_area(gray_area, color_area, structure=True):
    """
    gray_area : lista de 9 tuplas (y, x, val_gris)
    color_area: lista de 9 tuplas (y, x, [B,G,R])
    structure : True => parásito, False => no-parásito

    Retorna un dict con TODAS las features calculadas + 'structure'.
    """
    if len(gray_area) != 9:
        raise ValueError(f"Parche gris inválido: len={len(gray_area)}")

    # 1. Construir array 3x3 en gris
    subimage_vals = [item[2] for item in gray_area]  # val_gris
    subimage_gray = np.array(subimage_vals).reshape(3, 3)

    # 2. Features en gris
    intensity_feat  = calculate_intensity_features(subimage_gray)
    glcm_feat       = calculate_glcm_features(subimage_gray)
    lbp_feat        = calculate_lbp_features(subimage_gray)
    haralick_feat   = calculate_haralick_features(subimage_gray)
    geometry_feat   = calculate_geometry_features(subimage_gray)

    # FFT
    fft_feat = calculate_fft_features(subimage_gray)

    # Wavelet
    wavelet_feat = calculate_wavelet_features(subimage_gray, wavelet='haar', level=1)

    # Nuevas categorías: Gradiente
    gradient_feat   = calculate_gradient_features(subimage_gray)

    # Nuevas categorías: HOG
    try:
        hog_feat = calculate_hog_features(subimage_gray)
    except Exception as e:
        print(f"[WARNING] Fallo en HOG: {e}")
        hog_feat = {"hog_features": [0] * 81}  # Ajusta según el tamaño esperado

    # Nuevas categorías: Contornos
    contour_feat    = calculate_contour_features(subimage_gray)

    # 3. Extraer características de color
    color_feat = {}
    color_hist_feat = {}
    if len(color_area) == 9:
        color_feat = calculate_color_features_from_patch(color_area)
        color_hist_feat = calculate_color_histogram_features(color_area)  # Ahora devuelve características individuales

    # Nuevas categorías: Gabor
    gabor_feat = calculate_gabor_features(subimage_gray)

    # 4. Unir
    features = {
        **intensity_feat,
        **glcm_feat,
        **lbp_feat,
        **haralick_feat,
        **geometry_feat,
        **fft_feat,
        **wavelet_feat,
        **color_feat,
        **gradient_feat,
        **hog_feat,
        **contour_feat,
        **color_hist_feat,
        **gabor_feat,
        "structure": 1 if structure else 0  # 1=parásito, 0=no-parásito
    }
    
    return features

# ==========================
# 4. Función para recorrer TODOS los parches y generar la lista de features
# ==========================
def process_all_areas(parasite_areas_gray, parasite_areas_color,
                      non_parasite_areas_gray_local, non_parasite_areas_color_local,
                      non_parasite_areas_gray_external, non_parasite_areas_color_external):
    features_list = []

    # Procesar parches de cada categoría
    for gray_area, color_area in zip(parasite_areas_gray, parasite_areas_color):
        feats = extract_features_for_area(gray_area, color_area, structure=True)
        if feats is not None:
            features_list.append(feats)

    for gray_area, color_area in zip(non_parasite_areas_gray_local, non_parasite_areas_color_local):
        feats = extract_features_for_area(gray_area, color_area, structure=False)
        if feats is not None:
            features_list.append(feats)

    for gray_area, color_area in zip(non_parasite_areas_gray_external, non_parasite_areas_color_external):
        feats = extract_features_for_area(gray_area, color_area, structure=False)
        if feats is not None:
            features_list.append(feats)

    return features_list


# ==========================
# 5. Función para estandarizar (opcional) y generar un DataFrame final
# ==========================
def build_and_scale_features_df(features_list, do_scale=True):
    """
    Toma la lista de features (list of dicts) y la convierte a un DataFrame.
    Si do_scale=True, estandariza las columnas numéricas (excepto 'structure').

    Retorna un DataFrame final con los features escalados (y la columna 'structure').
    """
    # Convertimos la lista de dicts a un DataFrame
    df = pd.DataFrame(features_list)

    # Asegurarnos de que existe la columna 'structure'
    if 'structure' not in df.columns:
        df['structure'] = 0  # Por si acaso

    # Elegimos las columnas numéricas que deseamos escalar
    # Excluimos 'structure' y, si quieres, LBP hist que es una lista
    # Aquí detectamos con df.select_dtypes, pero a veces
    # conviene enumerar manualmente
    numeric_cols = []
    for col in df.columns:
        # Ignorar la columna 'structure' (etiqueta)
        if col == 'structure':
            continue
        # Ignorar si es un array/list (caso 'lbp_hist')
        if isinstance(df[col].iloc[0], list):
            # lbp_hist es lista => la dejamos sin escalar, o la procesarías aparte
            continue
        # De lo contrario, la consideramos numérica
        numeric_cols.append(col)

    if do_scale:
        scaler = StandardScaler()
        df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    return df



# Suponemos que todas las funciones auxiliares y definiciones de características ya están incluidas.
# (calculate_intensity_features, calculate_glcm_features, etc.)
##############################################################################
#                          Etapa 4.2: Pipeline Completo                      #
##############################################################################
def example_run_stage4_2(processed_images, annotations_csv, roi_size=44, visualize=False, output_csv="features_stage4_2.csv"):
    """
    Pipeline consolidado de la etapa 4.2.
      - Llama a la etapa 4.1 para procesar todas las imágenes y ROIs.
      - Extrae características de los parches (gris y color).
      - Genera un DataFrame con las características escaladas y lo guarda en un CSV.

    Parámetros:
    -----------
    processed_images : dict
        Diccionario con imágenes procesadas (de la etapa 1).
    annotations_csv : str
        Ruta al archivo CSV con anotaciones.
    roi_size : int
        Tamaño de los ROIs.
    visualize : bool
        Si es True, visualiza los resultados.
    output_csv : str
        Ruta del archivo CSV donde se guardarán los resultados.

    Retorna:
    --------
    df_features : pd.DataFrame
        DataFrame final con las características extraídas.
    """

    # Paso 1: Ejecutar la etapa 4.1
    results_4_1 = pipeline_stage_4_1(
        processed_images=processed_images,
        annotations_csv_path=annotations_csv,
        roi_size=roi_size,
        visualize=visualize
    )
    # Imprimir valores en consola
    print_patch_values(results_4_1)
    
    # Paso 2: Iterar por las imágenes y ROIs
    all_features = []

    for img_name, rois_results in results_4_1.items():
        print(f"\n=== Procesando imagen: {img_name} ===")
        for roi_idx, roi_data in enumerate(rois_results, start=1):
            print(f"--- ROI #{roi_idx} ---")

            # Extraer listas de parches
            parasite_areas_gray = roi_data["parasite_areas_gray"]
            non_parasite_areas_gray_local = roi_data["non_parasite_areas_gray_local"]
            non_parasite_areas_gray_external = roi_data["non_parasite_areas_gray_external"]

            parasite_areas_color = roi_data["parasite_areas_color"]
            non_parasite_areas_color_local = roi_data["non_parasite_areas_color_local"]
            non_parasite_areas_color_external = roi_data["non_parasite_areas_color_external"]

            # Validar que los parches gris y color correspondan
            assert len(parasite_areas_gray) == len(parasite_areas_color), "Mismatch entre parches gris y color (parásitos)."
            assert len(non_parasite_areas_gray_local) == len(non_parasite_areas_color_local), "Mismatch entre parches gris y color (no-parásitos locales)."
            assert len(non_parasite_areas_gray_external) == len(non_parasite_areas_color_external), "Mismatch entre parches gris y color (no-parásitos externos)."

            # Extraer características para todos los parches
            roi_features = process_all_areas(
                parasite_areas_gray,
                parasite_areas_color,
                non_parasite_areas_gray_local,
                non_parasite_areas_color_local,
                non_parasite_areas_gray_external,
                non_parasite_areas_color_external
            )

            print("Piwicho - error found here")
            # Agregar al conjunto general
            all_features.extend(roi_features)

    # Paso 3: Crear un DataFrame final
    df_features = build_and_scale_features_df(all_features, do_scale=True)

    # Paso 4: Guardar en un archivo CSV
    df_features.to_csv(output_csv, index=False)
    print(f"\n[Etapa 4.2] Resultados guardados en: {output_csv}")
    #df_features = "2"

    return df_features

# ==========================
# Ejemplo de uso (Etapa 4.2)
# ==========================

# Directorio de entrada
image_dir = "./img"  # Cambia esta ruta según tu estructura de directorios

# Parámetros
target_size = (768, 1024)  # Dimensiones estándar
clahe_params = (4, (16, 16))  # Parámetros de CLAHE

# Ejecutar el preprocesamiento
processed_images = preprocess_images_in_directory(image_dir, target_size, clahe_params)

annotations_csv = "././ft_orig/combined_data.csv"

df_final = example_run_stage4_2(processed_images, annotations_csv, roi_size=44, visualize=True)




# Configuramos pandas para mostrar todas las filas y columnas
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# Ahora imprimimos el DataFrame
#print(df_final)

# En este df_features ya tienes tus features escalados y la columna 'structure'.
# Listo para tu entrenamiento de ML.



Imágenes procesadas: 4489
Imágenes problemáticas: 0
Procesando ROIs para la imagen: field0001.jpg
Procesando ROIs para la imagen: field0004.jpg
Procesando ROIs para la imagen: field0005.jpg
Procesando ROIs para la imagen: field0006.jpg
Procesando ROIs para la imagen: field0007.jpg
Procesando ROIs para la imagen: field0008.jpg
Procesando ROIs para la imagen: field0009.jpg
Procesando ROIs para la imagen: field0010.jpg
Procesando ROIs para la imagen: field0011.jpg
Procesando ROIs para la imagen: field0012.jpg
Procesando ROIs para la imagen: field0014.jpg
Procesando ROIs para la imagen: field0015.jpg
Procesando ROIs para la imagen: field0016.jpg
Procesando ROIs para la imagen: field0017.jpg
Procesando ROIs para la imagen: field0018.jpg
Procesando ROIs para la imagen: field0019.jpg
Procesando ROIs para la imagen: field0020.jpg
Procesando ROIs para la imagen: field0021.jpg
Procesando ROIs para la imagen: field0022.jpg
Procesando ROIs para la imagen: field0023.jpg
Procesando ROIs para la imag